# Imports

To refresh imports

In [1]:
%reload_ext autoreload
%autoreload 2

Imports

In [2]:
# -------------------------------- torch stuff ------------------------------- #
import torch
from torch.utils.data import Dataset, DataLoader


# ----------------------------------- other ---------------------------------- #
from glob import glob
# import soundfile as sf
from scipy.io import wavfile
import json
import os
from tqdm import tqdm
import sys
import wandb

# ---------------------------------- Custom ---------------------------------- #
from utils.precision_loss import show_sum_error

### Why using scipy.io instead soundfile

In [3]:
# print("soundfile converts directly to float 64")
# sf.read("../data/LibriCount/10_85b5ac.wav")[0]

In [4]:
print("wavfile concervs the original format : int16")
wavfile.read("../data/LibriCount/10_85b5ac.wav")[1]

wavfile concervs the original format : int16


array([ 3095,  3265,  4328, ..., -1835, -1613, -1282], dtype=int16)

### Format

In [5]:
F16 = torch.float16
F32 = torch.float32
F64 = torch.float64

In [6]:
show_sum_error()

Conversion Errors from int16 in range(-32768, 32768) maxed by (max is=) :
	Float 16  8.0
	Float 32  0.0
	Float 64  0.0


# Consts

In [7]:
data_dir = "../data/LibriCount"
FTYPE = F32

# Dataset

In [8]:
class AudioCountGender(Dataset):
    def __init__(self, data_dir=data_dir, dtype = FTYPE, cache=True):
        self.sounds = glob(os.path.join(data_dir,"*.wav"))
        self.labels = glob(os.path.join(data_dir,"*.json"))
        self.dtype = dtype
        self.cache = cache
        if self.cache:
            self.data = []
            for index in range(len(self.sounds)):
                sample_rate, clip = wavfile.read(self.sounds[index])
                with open(self.labels[index]) as f:
                    label = json.load(f)
                genders = [0, 0] #[Male, Female]
                for person in label:
                    gender = person["sex"]
                    if gender == "F":
                        genders[1] += 1
                    else :
                        genders[0] += 1
                self.data.append([torch.tensor(clip, dtype=self.dtype).unsqueeze(0), torch.tensor(genders, dtype=self.dtype)]) #unsqueeze serves for channel = 1
                
                
    def __getitem__(self, index):
        if self.cache:
            return self.data[index]
        else:
            # clip, sample_rate = sf.read(self.sounds[index])
            sample_rate, clip = wavfile.read(self.sounds[index])
            with open(self.labels[index]) as f:
                label = json.load(f)
            genders = [0, 0] #[Male, Female]
            for person in label:
                gender = person["sex"]
                if gender == "F":
                    genders[1] += 1
                else :
                    genders[0] += 1
            return torch.tensor(clip, dtype=self.dtype).unsqueeze(0), torch.tensor(genders, dtype=self.dtype) #unsqueeze serves for channel = 1
    def __len__(self):
        return len(self.sounds)

In [9]:
data = AudioCountGender()
data[800]

[tensor([[4939., 3212., 1816.,  ..., 7072., 8158., 6819.]]), tensor([4., 3.])]

# Dataloader

In [10]:
BATCH_SIZE = 128
dataloader = DataLoader(dataset=data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

# Measures

In [11]:
from time import time
# L = []
start = time()
for d in dataloader:
    # L.append(d)
    print(len(d[0]), end="_")
end = time()
print()
print("Duration: {:.2f}s".format(end-start))
#Energy efficient mode
#Duration: 55.97s for F16 and no caching and list appending
#Duration: 1.72s for F16 and caching and list appending
#Equilibre mode 
#Duration: 12.46s for F16 and no caching and no list appending
#Duration: 0.88s for F16 and caching andn no list appending

128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_88_
Duration: 0.76s


# Simple model

In [12]:
# torch sequential
c_1 = 64
kernel_1 = 21
c_2 = 64
kernel_2 = 7
kernel_pool = 3

c_3 = 128
kernel_3 = 3
c_4 = 256
kernel_4 = 3
model = torch.nn.Sequential( #input size = 80000
    torch.nn.Conv1d(1, c_1, kernel_1, stride=5, dtype=FTYPE),
    torch.nn.Conv1d(c_1, c_2, kernel_2, dtype=FTYPE),
    torch.nn.ReLU(),
    torch.nn.Dropout(),
    torch.nn.MaxPool1d(kernel_pool),
    
    torch.nn.Conv1d(c_2, c_3, kernel_3, dtype=FTYPE),
    torch.nn.Conv1d(c_3, c_4, kernel_4, dtype=FTYPE),
    torch.nn.ReLU(),
    torch.nn.Dropout(),
    torch.nn.MaxPool1d(kernel_pool),
    
    torch.nn.Flatten(),
    torch.nn.Linear(c_4*1775, 2, dtype=FTYPE),
)

In [13]:
example = d[0].to("cuda")
model.to("cuda")

print("Shape : ", model.forward(example).shape)
model.forward(example)

Shape :  torch.Size([88, 2])


tensor([[ 2.2570e+02,  5.5099e+02],
        [-1.2534e+03,  1.3617e+02],
        [-4.6795e+01,  7.4459e+02],
        [-8.2248e+02,  8.5553e+02],
        [ 3.1124e+02,  3.1932e+02],
        [-5.9964e+02,  1.2370e+03],
        [-3.0364e+02,  1.5586e+02],
        [-6.4023e+02,  1.3432e+03],
        [ 5.3624e+02, -1.4660e+02],
        [-3.1876e+02, -8.3244e+01],
        [-6.2716e+02, -8.6650e+02],
        [-6.2086e+00,  9.5093e+01],
        [-3.5492e+02, -4.7577e+02],
        [-1.3043e+03, -3.5862e+02],
        [-3.1223e+02, -1.3156e+02],
        [ 1.5397e+00,  1.9312e+00],
        [ 2.8877e+02,  1.2216e+03],
        [-8.6916e+02,  1.0431e+03],
        [-5.5560e+02,  7.5136e+02],
        [-6.3094e+02,  5.6613e+02],
        [-3.3878e+00, -2.9167e+00],
        [-3.1200e-01, -1.0904e+01],
        [ 1.1385e+03,  2.6615e+02],
        [-7.8303e+02,  7.8648e+01],
        [ 4.6866e+02,  4.2235e+02],
        [-1.4756e+03, -1.0930e+03],
        [-7.7822e+01,  2.3077e+02],
        [ 3.6051e+02,  8.926

# Wandb

In [14]:
wandb.init(project="klee_project_audio", entity="mustapha")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mustapha (use `wandb login --relogin` to force relogin)


In [15]:
LEARNING_RATE = 1e-2
EPOCHS = 700
OPTIMIZER_L2 = 0.01
OPTIMIZER_MOM = 0.9
SCHEDULER_PATIENCE = 10
SCHEDULER_FACTOR = 0.5
SCHEDULER_MIN_LR = 1e-6
CLIP_GRAD = 1e5


wandb.config.update({
    "learning_rate": LEARNING_RATE,
    "epochs": EPOCHS,
    "OPTIMIZER": "SGD",
    "OPTIMIZER_L2":OPTIMIZER_L2,
    "OPTIMIZER_MOM": OPTIMIZER_MOM,
    "batch_size": BATCH_SIZE,
    "SCHEDULER": "ReduceLROnPlateau",
    "SCHEDULER_PATIENCE": SCHEDULER_PATIENCE,
    "SCHEDULER_FACTOR": SCHEDULER_FACTOR,
    "SCHEDULER_MIN_LR": SCHEDULER_MIN_LR,
    "CLIP_GRAD": CLIP_GRAD,
})

# Simple train

In [ ]:
loss = torch.nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, weight_decay=OPTIMIZER_L2, momentum=OPTIMIZER_MOM)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=SCHEDULER_PATIENCE, factor=SCHEDULER_FACTOR,min_lr=SCHEDULER_MIN_LR)

model.to("cuda")

for epoch in range(EPOCHS):
    mean_loss = 0
    for d in tqdm(dataloader):
        audios = d[0].to("cuda")
        labels = d[1].to("cuda")
        predictions = model.forward(audios)
        loss_value = loss(predictions, labels)
        mean_loss += loss_value.item()
        optimizer.zero_grad()
        loss_value.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), -CLIP_GRAD, CLIP_GRAD)
        optimizer.step()
    scheduler.step(mean_loss/len(dataloader))
    print("Epoch {}/{}".format(epoch+1, EPOCHS))
    print("Train Loss : {:.4f}".format(mean_loss/len(dataloader)))
    wandb.log({"loss":mean_loss/len(dataloader), "epoch":epoch, "next_lr" :scheduler.state_dict()["_last_lr"][0]})
    # wandb.watch(model)

100%|██████████| 45/45 [00:17<00:00,  2.55it/s]


Epoch 1/700
Train Loss : 266135.1007


100%|██████████| 45/45 [00:17<00:00,  2.54it/s]


Epoch 2/700
Train Loss : 176731.8321


100%|██████████| 45/45 [00:17<00:00,  2.54it/s]


...

100%|██████████| 45/45 [00:17<00:00,  2.50it/s]


Epoch 659/700
Train Loss : 9.8346


100%|██████████| 45/45 [00:18<00:00,  2.49it/s]


Epoch 660/700
Train Loss : 9.8367


 11%|█         | 5/45 [00:02<00:16,  2.49it/s]